# Predicting sentiment on the IMDB dataset

In [1]:
!date

Fri Dec 15 17:15:32 CET 2017


In this notebook, we show you how to train an RNN to classify movie review sentences. We mostly start from scratch, so that you should be able to plug in your own dataset without too much hassle. Furthermore, we explain some best practices with skorch, and how to perform a randomized hyper-parameter search.

## Import

In [2]:
import os
import tarfile

In [3]:
import numpy as np
from scipy import stats
from sklearn.datasets import load_files
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import RandomizedSearchCV
from dstoolbox.transformers import Padder2d
from skorch import NeuralNetClassifier
import torch
from torch import nn
F = nn.functional

## Constants

In [4]:
VOCAB_SIZE = 1000  # This is on the low end
MAX_LEN = 50  # Texts are pretty long on average, this is on the low end
USE_CUDA = True  # Set this to False if you don't want to use CUDA
NUM_CV_STEPS = 10  # Number of randomized search steps to perform

## Preparation

First make sure to install an additional package, dstoolbox

    $ pip install dstoolbox

Also, download the IMDB dataset

    $ wget http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz

## Load data

### Untar and unzip data

In [5]:
if not os.path.exists('aclImdb'):
    # unzip data if it does not exist
    with tarfile.open('aclImdb_v1.tar.gz', 'r:gz') as f:
        f.extractall()

### Description of the dataset

In [6]:
!head -n 8 aclImdb/README

Large Movie Review Dataset v1.0

Overview

This dataset contains movie reviews along with their associated binary
sentiment polarity labels. It is intended to serve as a benchmark for
sentiment classification. This document outlines how the dataset was
gathered, and how to use the files provided. 


In [7]:
!tail -n 22 aclImdb/README

@InProceedings{maas-EtAl:2011:ACL-HLT2011,
  author    = {Maas, Andrew L.  and  Daly, Raymond E.  and  Pham, Peter T.  and  Huang, Dan  and  Ng, Andrew Y.  and  Potts, Christopher},
  title     = {Learning Word Vectors for Sentiment Analysis},
  booktitle = {Proceedings of the 49th Annual Meeting of the Association for Computational Linguistics: Human Language Technologies},
  month     = {June},
  year      = {2011},
  address   = {Portland, Oregon, USA},
  publisher = {Association for Computational Linguistics},
  pages     = {142--150},
  url       = {http://www.aclweb.org/anthology/P11-1015}
}

References

Potts, Christopher. 2011. On the negativity of negation. In Nan Li and
David Lutz, eds., Proceedings of Semantics and Linguistic Theory 20,
636-659.

Contact

For questions/comments/corrections please contact Andrew Maas
amaas@cs.stanford.edu


### Read in data

In [8]:
dataset = load_files('aclImdb/train/', categories=['pos', 'neg'])

In [9]:
dataset.keys()

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])

### Only minimal data transformation

We mostly leave the data as is; for better reults, we should for instance remove markup, but we leave this out for brevity.

In [10]:
X, y = dataset['data'], dataset['target']
X = np.asarray([x.decode() for x in X])  # decode from bytes

### A peak at the data

In [11]:
for text, target in zip(X[:3], y):
    print("Target: {}".format(dataset['target_names'][target]))
    print(text)
    print()

Target: pos
Zero Day leads you to think, even re-think why two boys/young men would do what they did - commit mutual suicide via slaughtering their classmates. It captures what must be beyond a bizarre mode of being for two humans who have decided to withdraw from common civility in order to define their own/mutual world via coupled destruction.<br /><br />It is not a perfect movie but given what money/time the filmmaker and actors had - it is a remarkable product. In terms of explaining the motives and actions of the two young suicide/murderers it is better than 'Elephant' - in terms of being a film that gets under our 'rationalistic' skin it is a far, far better film than almost anything you are likely to see. <br /><br />Flawed but honest with a terrible honesty.

Target: neg
Words can't describe how bad this movie is. I can't explain it by writing only. You have too see it for yourself to get at grip of how horrible a movie really can be. Not that I recommend you to do that. There 

## Transformation steps

There are many possible ways to transform our data so that we can pass it to our neural net. What we effectively need is to transform a list of strings to an array of indices, where each row corresponds to one sample, and in each row, each int represents a specific word.

Below we show one way to achieve this that leverages sklearn's `CountVectorizer`. Here is its description from the sklearn documentation:

*Convert a collection of text documents to a matrix of token counts*

*This implementation produces a sparse representation of the counts using scipy.sparse.csr_matrix.*

At first glance, it would seem that this is not what we need, but we can easily extract the indices for each row of the sparse matrix by calling `.nonzero()`. This is shown in the function below, which we will wrap into an sklearn `FunctionTransformer` later:

In [12]:
def counts_to_indices(Xs):
    return np.asarray([x.nonzero()[1] for x in Xs])

Furthermore, by using sklearn instead of rolling our own transformation code, we gain the following benefits:

* battle-tested, (mostly) bug free code
* since it is an sklearn transformer, we can put it into a `Pipeline`
* many parameters for us to test in a hyper-parameter search

For more on the last point, see the section about randomized search.

Finally, we have to solve a small problem, namely that sentences have different number of words. This results in a heterogenous array but we need a homogeneous array. With the help of dstoolbox's `Padder2d`, we get this functionality in an sklearn transformer. (Note: We set `pad_value=VOCAB_SIZE` to give the padded value a unique index, since the other indices will range from 0 to VOCAB_SIZE-1).

Putting all of this together, these are the transformation steps in the pipeline before the actual neural net:

In [13]:
steps = [
    ('counts', CountVectorizer(max_features=VOCAB_SIZE)),
    ('to_idx', FunctionTransformer(counts_to_indices, accept_sparse=True)),
    ('pad', Padder2d(max_len=MAX_LEN, pad_value=VOCAB_SIZE, dtype=int)),
]

Here is what the output looks like:

In [14]:
Pipeline(steps).fit_transform(X[:3])

array([[ 92, 177, 209,  91,  75, 163, 106,  14,  13,   9,  70, 168, 155,
        144, 194,  79, 179,  72,  61, 178,  25, 127,   2, 123,  68, 176,
        154, 161,  85,   3,  12,  73, 180, 188, 122,  80,  31, 124, 149,
        132, 101,  30,  53,  46, 214, 146,  51, 140,  98,  38],
       [ 64,  69, 125, 192, 135, 145,  37,  59, 211, 164,  50,  66, 204,
         82,  20, 167,  84, 134, 201,  10,  60,  45, 111, 121, 162, 126,
         22, 189,  58,  21,  86, 172, 152, 198, 110,  28, 141, 147,   1,
        197,  99,   5,  71,  42,   6,  11, 165, 114,  55, 150],
       [100,   0,  15, 200,  96,  34, 136, 138,   4, 174, 143, 202, 196,
        166, 116, 139,  16,  81,  88, 157, 195,  62,  56, 105,  90, 109,
         36,  24, 131, 108, 203, 153, 148, 151,  65,  82, 134, 201,  11,
        103,  78, 187,  33,  79, 179,  25,  85, 180,  31, 124]])

As desired, we have a homogeneous array of indices, exactly what we need.

## The RNN

We define a rather simple RNN with just embeddings, a single recurrent layer, and an output layer. To be later able to test all hyper-parameters, we make sure to pass them to the `__init__` of our pytorch module.

In [15]:
class ImdbNet(nn.Module):
    def __init__(
            self,
            embedding_dim=128,
            rec_layer_type='lstm',
            num_units=128,
            dropout=0,
    ):
        super().__init__()
        self.embedding_dim = embedding_dim
        self.rec_layer_type = rec_layer_type.lower()
        self.num_units = num_units
        self.dropout = dropout

        self.emb = nn.Embedding(VOCAB_SIZE + 1, embedding_dim=self.embedding_dim)
        
        rec_layer = {'lstm': nn.LSTM, 'gru': nn.GRU}[self.rec_layer_type]
        # We have to make sure that the recurrent layer is batch_first,
        # since sklearn assumes the batch dimension to be the first
        self.rec = rec_layer(self.embedding_dim, self.num_units, batch_first=True)

        self.output = nn.Linear(self.num_units, 2)

    def forward(self, X):
        embeddings = self.emb(X)
        # from the recurrent layer, only take the activities from the last sequence step
        if self.rec_layer_type == 'gru':
            _, rec_out = self.rec(embeddings)
        else:
            _, (rec_out, _) = self.rec(embeddings)
        # dim 0 is seq dimension -> squeeze gets rid of it and thus makes the tensor batch_first
        rec_out = rec_out.squeeze(0)
        drop = F.dropout(rec_out, p=self.dropout)
        # Remember that the final non-linearity should be softmax, so that our predict_proba
        # method outputs actual probabilities!
        out = F.softmax(self.output(drop))
        return out

We wrap the pytorch module into a skorch `NeuralNetClassifier`, since we are dealing with a binary classification task, and append the step to our transformation steps.

In [16]:
steps.append(
    ('net', NeuralNetClassifier(
        ImdbNet,
        use_cuda=USE_CUDA,
        max_epochs=5,
        lr=0.01,
        optimizer=torch.optim.RMSprop,
    ))
)

Now we are good to go:

In [17]:
pipe = Pipeline(steps)

In [18]:
pipe.fit(X, y)

/home/bbossan_dev/anaconda3/envs/skorch/lib/python3.6/site-packages/ipykernel_launcher.py:34: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        0.6960       0.7590        0.5355  3.7855
      2        0.4711       0.7900        0.4737  3.5145
      3        0.3972       0.7958        0.4542  3.0171
      4        0.3495       0.8026        0.4699  3.7981
      5        0.3098       0.7684        0.5149  3.6162


Pipeline(memory=None,
     steps=[('counts', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=1000, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        str...ec): LSTM(128, 128, batch_first=True)
    (output): Linear(in_features=128, out_features=2)
  ),
))])

## Randomized search

The results above were already okay, but we have many hyper-parameters, so of course we would like to know which ones are best. Therefore, we perform a randomized search. For those not aware, a randomized search is like a grid search, but instead of testing the parameters systematically, they are drawn randomly from a distribution. In practice, compared to grid search, randomized search will find you better parameter values in a shorter amount of time.

For the randomized search, we turn off the net's verbosity to not clutter the notebook. Also, we set `train_split=None`. This is because we don't need an internal train/valid split, given that sklearn's `RandomizedSearchCV` already takes care of cross-validation.

In [19]:
pipe.set_params(net__verbose=0, net__train_split=None)

Pipeline(memory=None,
     steps=[('counts', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=1000, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        str...ec): LSTM(128, 128, batch_first=True)
    (output): Linear(in_features=128, out_features=2)
  ),
))])

Now we would like to set the hyper-parameter range to test. With randomized search, we can either specify a list (mostly for discreet variables) or a scipy stats distribution, from which sklearn will sample automatically.

As we can see below, we can extend the randomized search to not only cover the parameters we defined for our RNN, but also to cover the way we construct our vocabulary using sklearn's `CountVectorizer`. This is why we said earlier that we should use it instead of implementing it ourselves. As shown below, we test:

* stop_words: whether to remove english stop words or not
* lowercase: whether to turn all words lower-cased or not
* ngram_range: whether to use word uni-grams or bi-grams

Additionally, we test some hyper-parameters on the RNN module itself (e.g. LSTM vs GRU) and on the skorch `NeuralNetClassifier` (e.g. `max_epochs`).

In [20]:
params = {
    'counts__stop_words': ['english', None],
    'counts__lowercase': [False, True],
    'counts__ngram_range': [(1, 1), (2, 2)],
    'net__module__embedding_dim': stats.randint(32, 256 + 1),
    'net__module__rec_layer_type': ['gru', 'lstm'],
    'net__module__num_units': stats.randint(32, 256 + 1),
    'net__module__dropout': stats.uniform(0, 0.9),
    'net__lr': [10**(-stats.uniform(1, 5).rvs()) for _ in range(NUM_CV_STEPS)],
    'net__max_epochs': [5, 10],
}

We define our randomized search and start fitting.

For demonstration purposes, we perform only a low number of iterations (10) and only fit on the first 1000 samples. Of course, with more time, we should try more steps and include all samples.

In [21]:
search = RandomizedSearchCV(pipe, params, n_iter=NUM_CV_STEPS, verbose=2, refit=False, scoring='accuracy')

In [22]:
%time search.fit(X[:1000], y[:1000])

Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV] counts__lowercase=True, counts__ngram_range=(1, 1), counts__stop_words=None, net__lr=0.0791673882203, net__max_epochs=5, net__module__dropout=0.378566015198, net__module__embedding_dim=80, net__module__num_units=88, net__module__rec_layer_type=gru 


/home/bbossan_dev/anaconda3/envs/skorch/lib/python3.6/site-packages/ipykernel_launcher.py:34: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


[CV]  counts__lowercase=True, counts__ngram_range=(1, 1), counts__stop_words=None, net__lr=0.0791673882203, net__max_epochs=5, net__module__dropout=0.378566015198, net__module__embedding_dim=80, net__module__num_units=88, net__module__rec_layer_type=gru, total=   1.0s
[CV] counts__lowercase=True, counts__ngram_range=(1, 1), counts__stop_words=None, net__lr=0.0791673882203, net__max_epochs=5, net__module__dropout=0.378566015198, net__module__embedding_dim=80, net__module__num_units=88, net__module__rec_layer_type=gru 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.5s remaining:    0.0s
/home/bbossan_dev/anaconda3/envs/skorch/lib/python3.6/site-packages/ipykernel_launcher.py:34: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


[CV]  counts__lowercase=True, counts__ngram_range=(1, 1), counts__stop_words=None, net__lr=0.0791673882203, net__max_epochs=5, net__module__dropout=0.378566015198, net__module__embedding_dim=80, net__module__num_units=88, net__module__rec_layer_type=gru, total=   1.0s
[CV] counts__lowercase=True, counts__ngram_range=(1, 1), counts__stop_words=None, net__lr=0.0791673882203, net__max_epochs=5, net__module__dropout=0.378566015198, net__module__embedding_dim=80, net__module__num_units=88, net__module__rec_layer_type=gru 


...

[CV]  counts__lowercase=True, counts__ngram_range=(1, 1), counts__stop_words=None, net__lr=6.4534767546e-06, net__max_epochs=5, net__module__dropout=0.691307357589, net__module__embedding_dim=190, net__module__num_units=217, net__module__rec_layer_type=gru, total=   1.1s
[CV] counts__lowercase=True, counts__ngram_range=(1, 1), counts__stop_words=None, net__lr=6.4534767546e-06, net__max_epochs=5, net__module__dropout=0.691307357589, net__module__embedding_dim=190, net__module__num_units=217, net__module__rec_layer_type=gru 


/home/bbossan_dev/anaconda3/envs/skorch/lib/python3.6/site-packages/ipykernel_launcher.py:34: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


[CV]  counts__lowercase=True, counts__ngram_range=(1, 1), counts__stop_words=None, net__lr=6.4534767546e-06, net__max_epochs=5, net__module__dropout=0.691307357589, net__module__embedding_dim=190, net__module__num_units=217, net__module__rec_layer_type=gru, total=   1.1s
CPU times: user 58.1 s, sys: 1.58 s, total: 59.7 s
Wall time: 1min


[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:  1.0min finished


RandomizedSearchCV(cv=None, error_score='raise',
          estimator=Pipeline(memory=None,
     steps=[('counts', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=1000, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        str...ec): LSTM(128, 128, batch_first=True)
    (output): Linear(in_features=128, out_features=2)
  ),
))]),
          fit_params=None, iid=True, n_iter=10, n_jobs=1,
          param_distributions={'counts__stop_words': ['english', None], 'counts__lowercase': [False, True], 'counts__ngram_range': [(1, 1), (2, 2)], 'net__module__embedding_dim': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fc0eef855f8>, 'net__module__rec_layer_type': ['gru', 'lstm'], 'net__modul...0.00010767701730012674, 0.00024224122540093021, 6.4534767545970891e-06], 'net__max_epochs': [5, 10]},
          pre_dispatch='2

Below we see the best accuracy we achieved and what the best hyper-parameters were. Of course the scores here are underwhelming, given that we used so few samples and iterations. Using all the data and trying out more iterations should lead to much better outcomes.

In [23]:
search.best_score_, search.best_params_

(0.53400000000000003,
 {'counts__lowercase': True,
  'counts__ngram_range': (1, 1),
  'counts__stop_words': None,
  'net__lr': 6.4534767545970891e-06,
  'net__max_epochs': 5,
  'net__module__dropout': 0.69130735758896922,
  'net__module__embedding_dim': 190,
  'net__module__num_units': 217,
  'net__module__rec_layer_type': 'gru'})